# 第五章 Pandas入门

## 数据结构

### 创建Series Dataframe 对象

传入嵌套字典

In [1]:
from pandas import Series,DataFrame
import pandas as pd
import numpy as np

In [2]:
pop={'Nevada':{2001:2.4,2002:2.9},'Ohio':{2000:1.5,2002:3.6}}
f1=DataFrame(pop)
f1

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,NaN
2002,2.9,3.6


### index的方法和属性<br>
方法|说明
:--|:--
`append` |连接两个 `index` ，产生一个**新的 Index**                                                   
`delete` |删除索引 `i` 处的元素，并得到**新的 Index**
`is_monotonic` |是否单调递增
`is_unique` |`index` 中没有重复项时，返回 `True`
`unique` |计算 index 中唯一值的数组

## 基本功能

### reindex

#### Series

In [3]:
obj=Series(['blue','yellow','red'],index=[0,2,4])
obj2=obj.reindex(range(6),method='ffill')               

#### DataFrame
传入一个列表，重新索引行<br>
传入两个 list ,分别重新索引行列<br>
重新索引列：`frame.reindex(columns=list)`

In [4]:
frame=DataFrame(np.arange(9).reshape(3,3),index=['a','b','c'],columns=['Ohio','Texas','California'])

**对行进行索引**

In [5]:
frame2=frame.reindex(['a','d','b','c'],method="ffill")  
frame2

,Ohio,Texas,California
a,0,1,2
d,6,7,8
b,3,4,5
c,6,7,8


※ **用 method 的前提：**`index must be monotonic increasing or decreasing`
<br>`method='ffill'`
<br>所以d行的值，取自c行：`frame['d']=frame['c']`

In [6]:
frame2.columns=[1,2,3]
frame3=frame2.reindex(columns=[1,2,3,4,5,5],method='ffill',limit=1)     #bfill 时返回 NaN
frame3.columns=['col1','col2','col3','col4','col5','col6']
frame3

,col1,col2,col3,col4,col5,col6
a,0,1,2,2,NaN,NaN
d,6,7,8,8,NaN,NaN
b,3,4,5,5,NaN,NaN
c,6,7,8,8,NaN,NaN


In [7]:
frame3.xs('col2',axis=1)  #xs方法，获得单行或单列

a    1
d    7
b    4
c    7
Name: col2, dtype: int32

**NOTES**
1. `columns` 为有序值时，可用 `method` 确认`fill` 方式。
2. 当传入两个列表同时 `reindex` 行列，`method` 只应用于 **行**
3. 调用 `limit` 时，`columns/index` 必须是单调的

### 丢弃值 `get/set values`

In [8]:
#丢弃行
frame3.drop('a')
#丢弃列
new_frame=frame3.drop('col6',axis=1)   #drop返回新的DataFrame对象

In [9]:
frame3.set_value('c','col5',-1)
frame3

,col1,col2,col3,col4,col5,col6
a,0,1,2,2,NaN,NaN
d,6,7,8,8,NaN,NaN
b,3,4,5,5,NaN,NaN
c,6,7,8,8,-1.0,NaN


### 填充
1. `reindex` `method` 填充： [`ffill`](#reindex)、`bfill`
2. 值填充，`fill_value`
3. `reindex` 时也可传入 `fill_value`

In [10]:
df1=DataFrame(np.arange(12).reshape(3,4),columns=list('abcd'))
df2=DataFrame(np.arange(20).reshape(4,5),columns=list('abcde'))
df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [17]:
df1.add(df2,fill_value=0)    #fill_value的值传入df1/df1的NaN位置，两个同时为NaN的位置，仍为NaN

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


**算术方法**


方法|说明
:--|:--
add|加
sub|减
mul|乘
div|除

### DataFrame与Series之间的运算
行广播运算(默认)
```python
df-series    #按行广播
```
列广播
```python
df.sub(series,axis=0)  #传入的轴是希望seriesd.index匹配的轴，匹配0轴后，在1轴上广播
```

### 函数应用与映射

In [12]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.595630,0.262811,-1.183760
Ohio,0.575663,1.056284,-0.933584
Texas,0.553856,-0.738137,0.463742
Oregon,0.120622,-0.442041,1.684424


**`apply`**方法<br>
作用在一行或一列上，默认0轴

In [13]:
f=lambda x: x.max()-x.min()
frame.apply(f)
frame.apply(f,axis=1)

Utah      1.779391
Ohio      1.989867
Texas     1.291993
Oregon    2.126465
dtype: float64

In [14]:
def f2(x):
    return Series([x.min(),x.max()],index=['min','max'])
frame.apply(f2)

,b,d,e
min,0.120622,-0.738137,-1.183760
max,0.595630,1.056284,1.684424


In [15]:
f3=lambda x: Series([x.min(),x.max()],index=['min','max'])
frame.apply(f3)

,b,d,e
min,0.120622,-0.738137,-1.183760
max,0.595630,1.056284,1.684424


**`applymap`**
<br>作用Series的单个元素上<br>
like doing `map(func, series)` for each series in the DataFrame
<br>e.g.内容格式化

In [16]:
format=lambda x: '%.2f' %x
frame.applymap(format)

,b,d,e
Utah,0.60,0.26,-1.18
Ohio,0.58,1.06,-0.93
Texas,0.55,-0.74,0.46
Oregon,0.12,-0.44,1.68


### 排序 `sort`
Refer: [numpy排序](ch04_01.ipynb#排序)
1. 按索引排序: 
```python
Series.sort_index()
Df.sort_index()
Df.sort_index(axis=1)
```
2. 按值排序
```python
Series.order()
Df.sort_index(by='column_name')   #will be deprecated
Df.sort_values(by=['a','b'])    #先按a排序，a值相同再按b排序
```

### 排名 `ranking`

返回元素在数组中的排名。<br>
相同值的处理：

method|说明
:--|:---
average|默认：Equal values are assigned a rank that is the average of the ranks of those values
min|使用最小排名
max|使用最大排名
first|按出现顺序排名

In [55]:
frame = DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                   'c': [-2, 5, 8, -2.5]})
frame.rank(axis=1,method='first')

,a,b,c
0,2.0,3.0,1.0
1,1.0,3.0,2.0
2,2.0,1.0,3.0
3,2.0,3.0,1.0


## 汇总和计算描述统计